# Scraping Data From A Web Page

This notebook scrapes tables from a specific Wikipedia page and stores them in this notebook's directory as an Excel workbook or CSV files.

#### 1. Import Python modules to access the necessary functions for converting web tables to Excel/CSV

In [ ]:
import requests  # HTTP requests to a chosen URL
from bs4 import BeautifulSoup  # HTML parsing & storage
import pandas as pd  # DataFrame creation of scraped tables
import os  # Operating system functionalities such as getting file paths
import xlsxwriter  # Excel file writing

#### 2. Hard-code variables for use by the WebScraper class

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'
markup = 'html'  # Markup language of web page
file_path = os.getcwd() + "\\"  # This notebook's file path
file_extensions = ['.csv', '.xlsx']  # Acceptable extensions for exporting
excel = True  # Exports Excel file (True) or CSVs (False)
excel_name = "Largest_US_Companies_by_Revenue"  # Excle file custom name
sheet_names = ['Public_Companies',
               'Private_Companies',
               'Companies_by_Profit']  # Custom names for Excel sheets or CSV file names

#### 3. Encapsulate all web scraping functions into a WebScraper class

In [ ]:
class WebScraper():
    # Assign values to instance variables when WebScraper object is initialized
    def __init__(self, url, markup, file_path, file_extensions,
                 excel=False, excel_name="workbook", sheet_names=['sheet']):
        self.url = url
        self.markup = markup
        self.file_path = file_path
        self.file_extensions = file_extensions
        self.excel = excel
        self.excel_name = excel_name
        self.sheet_names = sheet_names
        self.page = None
        self.soup = None
        self.df_list = []
    
    # Conduct HTTP Request and store resulting markup as text in the soup instance variable
    def MakeSoup(self):
        self.page = requests.get(self.url)
        self.soup = BeautifulSoup(self.page.text, self.markup)
        return self.soup
    
    # Scrape the soup variable for tables and store them in the dataframe list instance variable
    def BuildDataFrames(self):
        
        # Find all tables by the 'table' tag
        table_list = self.soup.find_all('table')
        
        # Iterate through the table list
        for table in table_list:
            
            # Find all headers of the current table by the 'th' tag
            raw_headers = table.find_all('th')
            
            # Skip Tables with zero headers & begin next iteration
            if len(raw_headers) == 0:
                continue
            
            # Remove markup tags & strip whitespace from each header
            clean_headers = [header.text.strip() for header in raw_headers]
            
            # Initiate a DataFrame with the clean headers
            df = pd.DataFrame(columns = clean_headers)

            # Find all rows of the current table by the 'tr' tag
            table_rows = table.find_all('tr')
            
            # Iterate through the table's rows
            for row in table_rows:

                # Find all values in the current row of the current table
                raw_row_data = row.find_all('td')
                
                # Skip invalid rows & begin next iteration
                if len(raw_row_data) == 0 or len(raw_row_data) != len(raw_headers):
                    continue
                
                # Remove markup tags & strip whitespace from each value
                clean_row_data = [value.text.strip() for value in raw_row_data]

                # Assign the current clean row's values to the end of the DataFrame
                # The last index is found by calculating the DataFrame's length
                length = len(df)  # Get length
                df.loc[length] = clean_row_data  # Assign clean row to end
            
            # Only adds non-empty DataFrames to the DataFrame list
            if len(df) > 0:
                self.df_list.append(df)
        
        return self.df_list
    
    # Build a list of sheet names for each DataFrame in the DataFrame list instance variable
    def BuildSheetNames(self):
        self.sheet_names = []
        i = 1
        while len(self.df_list) != len(self.sheet_names):
            self.sheet_names.append(f"sheet_{i}")
            i += 1
    
    # Export the DataFrames as an Excel workbook or CSV files
    def ExportDataFrames(self):
        
        # Build sheet names if its count does not match the DataFrame list's count
        if len(self.df_list) != len(self.sheet_names):
            self.BuildSheetNames()
        
        # Initialize an ExcelWriter object if user preference is to export as Excel
        if self.excel:
            # Writes & opens a new Excel workbook in the chosen File Path
            writer = pd.ExcelWriter(self.file_path + self.excel_name + self.file_extensions[1], engine='xlsxwriter')
        
        # Iterate through each DataFrame in the DataFrame list instance variable
        i = 0
        for df in self.df_list:
            # Writes current DataFrame to an Excel sheet or CSV file
            if self.excel:
                df.to_excel(writer, sheet_name=self.sheet_names[i], index=False)
            else:
                df.to_csv(self.file_path + self.sheet_names[i] + self.file_extensions[0], index = False)
            i += 1
        
        # Closes Excel workbook if Excel was chosen & opened by the ExcelWriter object
        if self.excel:
            writer.close()

#### 4. Initialize WebScraper object and run its functions to export web tables into Excel/CSV

In [ ]:
# Initialize WebScraper object with the hard-coded variables
webscraper = WebScraper(url, markup, file_path, file_extensions, 
                        excel, excel_name, sheet_names)

# Run the WebScraper functions
webscraper.MakeSoup()  # Gets markup & stores as soup
webscraper.BuildDataFrames()  # Converts markup tables into DataFrame objects
webscraper.ExportDataFrames()  # Exports DataFrames based on user preferred file extension